# 🧹 บทที่ 2.2: Data Cleaning - การทำความสะอาดข้อมูล

## 🎭 เรื่องราวของนักสืบข้อมูล

จากตอนที่แล้ว เราได้โหลดข้อมูลลูกค้า 500 คนมาแล้ว แต่เมื่อเช็คอีกครั้ง...

😱 **พบปัญหา!**
- บางคนไม่กรอก **อายุ** (Missing Values)
- มีคนรายงานอายุ **150 ปี**! (Outliers)
- มีคนกรอก Income เป็น **-50000** (Inconsistencies)

ถ้าใช้ข้อมูลสกปรกนี้เทรนโมเดล → โมเดลจะเรียนรู้ผิด!

**มาทำความสะอาดข้อมูลกันเถอะ!** 🧹✨

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')

print("✅ Libraries loaded!")

---

## 📊 สร้างข้อมูลที่มีปัญหา (Dirty Data)

เราจะจงใจสร้างข้อมูลที่มีปัญหาเพื่อฝึกแก้ไข

In [ ]:
np.random.seed(42)

# สร้างข้อมูลปกติ
n = 500
data = {
    'CustomerID': [f'C{str(i).zfill(4)}' for i in range(1, n+1)],
    'Age': np.random.randint(18, 65, n).astype(float),
    'Income': np.random.randint(25000, 120000, n).astype(float),
    'SpendingScore': np.random.randint(1, 100, n).astype(float),
    'PurchaseCount': np.random.randint(1, 50, n).astype(float)
}

df = pd.DataFrame(data)

# ========================================
# 🔥 จงใจทำให้ข้อมูลสกปรก!
# ========================================

# 1. Missing Values (15% ของข้อมูล)
missing_indices = np.random.choice(df.index, size=int(0.15*n), replace=False)
df.loc[missing_indices[:25], 'Age'] = np.nan
df.loc[missing_indices[25:50], 'Income'] = np.nan
df.loc[missing_indices[50:], 'SpendingScore'] = np.nan

# 2. Outliers (อายุแปลกๆ และ Income ผิดปกติ)
outlier_indices = np.random.choice(df.index, size=15, replace=False)
df.loc[outlier_indices[:5], 'Age'] = [150, 5, 200, 0, 250]  # อายุผิดปกติ
df.loc[outlier_indices[5:10], 'Income'] = [500000, 600000, 750000, 1000000, 850000]  # รายได้สูงผิดปกติ
df.loc[outlier_indices[10:], 'SpendingScore'] = [150, 200, -50, 300, 999]  # คะแนนผิดปกติ

# 3. Inconsistencies (ข้อมูลไม่สมเหตุสมผล)
inconsistent_indices = np.random.choice(df.index, size=10, replace=False)
df.loc[inconsistent_indices[:5], 'Income'] = [-10000, -50000, -25000, 0, -1000]  # รายได้ติดลบ
df.loc[inconsistent_indices[5:], 'PurchaseCount'] = [-5, -10, 0, -2, -8]  # จำนวนซื้อติดลบ

print("💀 Created DIRTY data with problems:")
print(f"   Total records: {len(df)}")
print(f"   ✓ Missing values")
print(f"   ✓ Outliers")
print(f"   ✓ Inconsistencies")
print("\nLet's investigate! 🔍")

---

## 🔍 Part 1: ตรวจสอบ Missing Values

### Missing Values คืออะไร?
- ข้อมูลที่หายไป (NaN, None, null)
- เกิดจาก: ลืมกรอก, ระบบ error, ข้อมูลไม่ครบ

### ทำไมต้องจัดการ?
- โมเดล ML ไม่ชอบ NaN → จะ error!
- Missing แบบสุ่ม → อาจทำให้โมเดล bias

In [ ]:
# ตรวจสอบ Missing Values
print("❓ Missing Values Check:")
print("="*50)

missing_count = df.isnull().sum()
missing_percent = (df.isnull().sum() / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing_count.index,
    'Missing Count': missing_count.values,
    'Percentage': missing_percent.values
})

print(missing_df[missing_df['Missing Count'] > 0])
print(f"\n📊 Total missing values: {df.isnull().sum().sum()}")

In [ ]:
# Visualize Missing Values
plt.figure(figsize=(12, 5))

# Heatmap
plt.subplot(1, 2, 1)
sns.heatmap(df.isnull(), cbar=True, yticklabels=False, cmap='viridis')
plt.title('Missing Values Heatmap', fontsize=14, fontweight='bold')
plt.xlabel('Columns')
plt.ylabel('Rows')

# Bar chart
plt.subplot(1, 2, 2)
missing_data = df.isnull().sum()[df.isnull().sum() > 0]
missing_data.plot(kind='bar', color='coral')
plt.title('Missing Values per Column', fontsize=14, fontweight='bold')
plt.xlabel('Column')
plt.ylabel('Number of Missing Values')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# วิธีแก้ไข Missing Values

print("🛠️ Solutions for Missing Values:\n")

# วิธีที่ 1: ลบทิ้ง (Drop)
df_dropped = df.dropna()
print(f"1️⃣ Drop rows with NaN:")
print(f"   Before: {len(df)} rows")
print(f"   After: {len(df_dropped)} rows")
print(f"   Lost: {len(df) - len(df_dropped)} rows ({((len(df) - len(df_dropped))/len(df)*100):.1f}%)")
print(f"   ⚠️ Use when: Missing < 5% of data\n")

# วิธีที่ 2: กรอกด้วยค่าเฉลี่ย (Mean)
df_mean = df.copy()
df_mean['Age'].fillna(df_mean['Age'].mean(), inplace=True)
df_mean['Income'].fillna(df_mean['Income'].mean(), inplace=True)
df_mean['SpendingScore'].fillna(df_mean['SpendingScore'].mean(), inplace=True)
print(f"2️⃣ Fill with Mean:")
print(f"   Age mean: {df['Age'].mean():.1f}")
print(f"   ✓ Good for: Normal distribution\n")

# วิธีที่ 3: กรอกด้วยค่ากลาง (Median)
df_median = df.copy()
df_median['Age'].fillna(df_median['Age'].median(), inplace=True)
df_median['Income'].fillna(df_median['Income'].median(), inplace=True)
df_median['SpendingScore'].fillna(df_median['SpendingScore'].median(), inplace=True)
print(f"3️⃣ Fill with Median:")
print(f"   Age median: {df['Age'].median():.1f}")
print(f"   ✓ Good for: Data with outliers\n")

# วิธีที่ 4: กรอกด้วยค่าที่เกิดบ่อยสุด (Mode)
df_mode = df.copy()
df_mode['SpendingScore'].fillna(df_mode['SpendingScore'].mode()[0], inplace=True)
print(f"4️⃣ Fill with Mode:")
print(f"   SpendingScore mode: {df['SpendingScore'].mode()[0]:.1f}")
print(f"   ✓ Good for: Categorical data\n")

# ใช้ Median (เหมาะสมที่สุดเพราะมี outliers)
df_clean = df_median.copy()
print("✅ We'll use MEDIAN filling (best for data with outliers)")

---

## 🎯 Part 2: ตรวจจับ Outliers

### Outliers คืออะไร?
- ข้อมูลที่ **ผิดปกติมาก** เมื่อเทียบกับข้อมูลอื่น
- ตัวอย่าง: อายุ 200 ปี, รายได้ 10 ล้าน/เดือน (ในข้อมูลพนักงานทั่วไป)

### ทำไมต้องจัดการ?
- Outliers ทำให้ค่าเฉลี่ยเพี้ยน
- โมเดลเรียนรู้ผิด → Prediction ไม่แม่นยำ

### วิธีหา Outliers:
1. **Z-Score** - ดูว่าห่างจากค่าเฉลี่ยกี่ standard deviation
2. **IQR (Interquartile Range)** - ใช้ช่วง Q1-Q3
3. **Visualization** - Box plot, Scatter plot

In [ ]:
# Visualize Outliers with Box Plots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Age
axes[0, 0].boxplot(df_clean['Age'].dropna(), vert=True)
axes[0, 0].set_title('Age Distribution (Before Cleaning)', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Age')
axes[0, 0].grid(axis='y', alpha=0.3)

# Income
axes[0, 1].boxplot(df_clean['Income'].dropna(), vert=True)
axes[0, 1].set_title('Income Distribution (Before Cleaning)', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Income ($)')
axes[0, 1].grid(axis='y', alpha=0.3)

# SpendingScore
axes[1, 0].boxplot(df_clean['SpendingScore'].dropna(), vert=True)
axes[1, 0].set_title('Spending Score (Before Cleaning)', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Score')
axes[1, 0].grid(axis='y', alpha=0.3)

# PurchaseCount
axes[1, 1].boxplot(df_clean['PurchaseCount'].dropna(), vert=True)
axes[1, 1].set_title('Purchase Count (Before Cleaning)', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Count')
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("👀 เห็นจุดที่ผิดปกติไหม? (จุดที่อยู่นอก whiskers)")

In [ ]:
# วิธีที่ 1: Z-Score Method
print("🔬 Method 1: Z-Score Detection")
print("="*60)
print("📌 Z-Score = (value - mean) / std")
print("📌 Outlier if |Z-Score| > 3\n")

def detect_outliers_zscore(df, column, threshold=3):
    mean = df[column].mean()
    std = df[column].std()
    z_scores = np.abs((df[column] - mean) / std)
    outliers = df[z_scores > threshold]
    return outliers, z_scores

# ตรวจสอบ Age
age_outliers, age_z = detect_outliers_zscore(df_clean, 'Age')
print(f"Age Outliers (Z-Score method): {len(age_outliers)} records")
print(f"Examples:")
print(age_outliers[['CustomerID', 'Age']].head())
print()

In [ ]:
# วิธีที่ 2: IQR Method (แนะนำ!)
print("📊 Method 2: IQR (Interquartile Range) Detection")
print("="*60)
print("📌 Q1 = 25th percentile")
print("📌 Q3 = 75th percentile")
print("📌 IQR = Q3 - Q1")
print("📌 Outlier if: value < Q1 - 1.5*IQR OR value > Q3 + 1.5*IQR\n")

def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    
    print(f"\n{column}:")
    print(f"   Q1: {Q1:.2f}")
    print(f"   Q3: {Q3:.2f}")
    print(f"   IQR: {IQR:.2f}")
    print(f"   Valid range: [{lower_bound:.2f}, {upper_bound:.2f}]")
    print(f"   Outliers found: {len(outliers)}")
    
    return outliers, lower_bound, upper_bound

# ตรวจสอบทุก column
age_out_iqr, age_lower, age_upper = detect_outliers_iqr(df_clean, 'Age')
income_out_iqr, income_lower, income_upper = detect_outliers_iqr(df_clean, 'Income')
score_out_iqr, score_lower, score_upper = detect_outliers_iqr(df_clean, 'SpendingScore')

In [ ]:
# แสดง Outliers ที่พบ
print("\n🚨 Found Outliers:")
print("="*60)

print("\n📍 Age Outliers:")
print(age_out_iqr[['CustomerID', 'Age']].sort_values('Age', ascending=False).head(10))

print("\n💰 Income Outliers:")
print(income_out_iqr[['CustomerID', 'Income']].sort_values('Income', ascending=False).head(10))

print("\n📊 Spending Score Outliers:")
print(score_out_iqr[['CustomerID', 'SpendingScore']].sort_values('SpendingScore', ascending=False).head(10))

In [ ]:
# แก้ไข Outliers
print("🛠️ Fixing Outliers:\n")

df_no_outliers = df_clean.copy()

# วิธีที่ 1: ตัดทิ้ง (Remove)
print("Method A: Remove outliers")
print(f"   Before: {len(df_no_outliers)} records")

# วิธีที่ 2: Cap/Clip (แนะนำ!)
print("\nMethod B: Cap outliers (Recommended) ✅")
print("   Replace values outside valid range with boundary values\n")

# Cap Age
df_no_outliers['Age'] = df_no_outliers['Age'].clip(lower=age_lower, upper=age_upper)
print(f"   Age capped to [{age_lower:.0f}, {age_upper:.0f}]")

# Cap Income
df_no_outliers['Income'] = df_no_outliers['Income'].clip(lower=income_lower, upper=income_upper)
print(f"   Income capped to [{income_lower:.0f}, {income_upper:.0f}]")

# Cap SpendingScore
df_no_outliers['SpendingScore'] = df_no_outliers['SpendingScore'].clip(lower=score_lower, upper=score_upper)
print(f"   SpendingScore capped to [{score_lower:.0f}, {score_upper:.0f}]")

print(f"\n✅ Outliers fixed! Records maintained: {len(df_no_outliers)}")

---

## ⚠️ Part 3: แก้ไข Inconsistencies

### Inconsistencies คืออะไร?
- ข้อมูลที่ **ไม่สมเหตุสมผล** ตาม business logic

### ตัวอย่าง:
- รายได้ **ติดลบ** (เป็นไปไม่ได้)
- จำนวนการซื้อ **ติดลบ** (ไม่มีทาง)
- อายุ **0 หรือ 200 ปี** (ผิดปกติ)
- SpendingScore **> 100 หรือ < 0** (คะแนนควรอยู่ 0-100)

In [ ]:
# หา Inconsistencies
print("🔍 Checking for Inconsistencies:\n")

# 1. รายได้ติดลบหรือ 0
negative_income = df_no_outliers[df_no_outliers['Income'] <= 0]
print(f"1️⃣ Negative or Zero Income: {len(negative_income)} records")
if len(negative_income) > 0:
    print(negative_income[['CustomerID', 'Income']].head())
    print("   ❌ Problem: Income cannot be negative or zero!\n")

# 2. จำนวนซื้อติดลบ
negative_purchase = df_no_outliers[df_no_outliers['PurchaseCount'] < 0]
print(f"2️⃣ Negative Purchase Count: {len(negative_purchase)} records")
if len(negative_purchase) > 0:
    print(negative_purchase[['CustomerID', 'PurchaseCount']].head())
    print("   ❌ Problem: Cannot have negative purchases!\n")

# 3. อายุไม่สมเหตุสมผล
weird_age = df_no_outliers[(df_no_outliers['Age'] < 18) | (df_no_outliers['Age'] > 100)]
print(f"3️⃣ Unrealistic Age: {len(weird_age)} records")
if len(weird_age) > 0:
    print(weird_age[['CustomerID', 'Age']].head())
    print("   ❌ Problem: Age should be 18-100 for adult customers!\n")

# 4. คะแนนนอกช่วง 0-100
weird_score = df_no_outliers[(df_no_outliers['SpendingScore'] < 0) | (df_no_outliers['SpendingScore'] > 100)]
print(f"4️⃣ Invalid Spending Score: {len(weird_score)} records")
if len(weird_score) > 0:
    print(weird_score[['CustomerID', 'SpendingScore']].head())
    print("   ❌ Problem: Score must be 0-100!\n")

In [ ]:
# แก้ไข Inconsistencies
print("🛠️ Fixing Inconsistencies:\n")

df_consistent = df_no_outliers.copy()

# 1. แก้รายได้ติดลบ → ใช้ Median
median_income = df_consistent[df_consistent['Income'] > 0]['Income'].median()
df_consistent.loc[df_consistent['Income'] <= 0, 'Income'] = median_income
print(f"✅ Fixed negative income → replaced with median: ${median_income:.0f}")

# 2. แก้จำนวนซื้อติดลบ → ใช้ 1 (ซื้อขั้นต่ำ)
df_consistent.loc[df_consistent['PurchaseCount'] < 0, 'PurchaseCount'] = 1
print(f"✅ Fixed negative purchases → replaced with minimum: 1")

# 3. แก้อายุผิดปกติ → ใช้ Median
median_age = df_consistent[(df_consistent['Age'] >= 18) & (df_consistent['Age'] <= 100)]['Age'].median()
df_consistent.loc[(df_consistent['Age'] < 18) | (df_consistent['Age'] > 100), 'Age'] = median_age
print(f"✅ Fixed unrealistic age → replaced with median: {median_age:.0f}")

# 4. แก้คะแนนนอกช่วง → Clip to 0-100
df_consistent['SpendingScore'] = df_consistent['SpendingScore'].clip(0, 100)
print(f"✅ Fixed spending score → clipped to [0, 100]")

print(f"\n🎉 All inconsistencies fixed!")

In [ ]:
# ตรวจสอบอีกครั้งหลังแก้ไข
print("🔍 Verification after cleaning:\n")

print("Age:")
print(f"   Min: {df_consistent['Age'].min():.1f}")
print(f"   Max: {df_consistent['Age'].max():.1f}")
print(f"   Mean: {df_consistent['Age'].mean():.1f}\n")

print("Income:")
print(f"   Min: ${df_consistent['Income'].min():.0f}")
print(f"   Max: ${df_consistent['Income'].max():.0f}")
print(f"   Mean: ${df_consistent['Income'].mean():.0f}\n")

print("SpendingScore:")
print(f"   Min: {df_consistent['SpendingScore'].min():.1f}")
print(f"   Max: {df_consistent['SpendingScore'].max():.1f}")
print(f"   Mean: {df_consistent['SpendingScore'].mean():.1f}\n")

print("PurchaseCount:")
print(f"   Min: {df_consistent['PurchaseCount'].min():.0f}")
print(f"   Max: {df_consistent['PurchaseCount'].max():.0f}")
print(f"   Mean: {df_consistent['PurchaseCount'].mean():.1f}\n")

print("✅ All values are now in valid ranges!")

---

## 📊 Before vs After Comparison

In [ ]:
# เปรียบเทียบก่อนและหลัง
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Age
axes[0, 0].hist(df['Age'].dropna(), bins=30, alpha=0.7, color='red', label='Before', edgecolor='black')
axes[0, 0].set_title('Age - Before Cleaning', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Age')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].legend()

axes[1, 0].hist(df_consistent['Age'], bins=30, alpha=0.7, color='green', label='After', edgecolor='black')
axes[1, 0].set_title('Age - After Cleaning', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Age')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].legend()

# Income
axes[0, 1].hist(df['Income'].dropna(), bins=30, alpha=0.7, color='red', label='Before', edgecolor='black')
axes[0, 1].set_title('Income - Before Cleaning', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Income ($)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].legend()

axes[1, 1].hist(df_consistent['Income'], bins=30, alpha=0.7, color='green', label='After', edgecolor='black')
axes[1, 1].set_title('Income - After Cleaning', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Income ($)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].legend()

# SpendingScore
axes[0, 2].hist(df['SpendingScore'].dropna(), bins=30, alpha=0.7, color='red', label='Before', edgecolor='black')
axes[0, 2].set_title('Spending Score - Before', fontsize=12, fontweight='bold')
axes[0, 2].set_xlabel('Score')
axes[0, 2].set_ylabel('Frequency')
axes[0, 2].legend()

axes[1, 2].hist(df_consistent['SpendingScore'], bins=30, alpha=0.7, color='green', label='After', edgecolor='black')
axes[1, 2].set_title('Spending Score - After', fontsize=12, fontweight='bold')
axes[1, 2].set_xlabel('Score')
axes[1, 2].set_ylabel('Frequency')
axes[1, 2].legend()

plt.tight_layout()
plt.show()

print("👀 เห็นความแตกต่างไหม? ข้อมูลหลังทำความสะอาดดูปกติมากขึ้น!")

In [ ]:
# บันทึกข้อมูลที่สะอาดแล้ว
df_consistent.to_csv('customer_data_clean.csv', index=False)
print("💾 Saved clean data to 'customer_data_clean.csv'")
print(f"\n📊 Final Statistics:")
print(f"   Total records: {len(df_consistent)}")
print(f"   Missing values: {df_consistent.isnull().sum().sum()}")
print(f"   All data is now clean and ready for ML! ✨")

---

## 📝 สรุป: Data Cleaning

### 🎯 สิ่งที่เราได้เรียนรู้:

#### 1. **Missing Values**
- **หา:** `df.isnull().sum()`
- **แก้:**
  - Drop: `df.dropna()`
  - Fill with mean: `df.fillna(df.mean())`
  - Fill with median: `df.fillna(df.median())` ← **แนะนำถ้ามี outliers**
  - Fill with mode: `df.fillna(df.mode()[0])`

#### 2. **Outliers**
- **หา:**
  - Z-Score: `|Z| > 3`
  - IQR: `Q1 - 1.5*IQR` ถึง `Q3 + 1.5*IQR` ← **แนะนำ**
- **แก้:**
  - Remove: ลบทิ้ง
  - Cap/Clip: จำกัดค่า ← **แนะนำ (รักษาจำนวนข้อมูล)**

#### 3. **Inconsistencies**
- **ตัวอย่าง:**
  - รายได้ติดลบ → ไม่เป็นไปได้
  - จำนวนซื้อติดลบ → ผิดตรรกะ
  - อายุ 0 หรือ 200 → ไม่สมจริง
- **แก้:**
  - ใช้ business logic กำหนดค่าที่ถูกต้อง
  - Replace ด้วย median/mean/mode
  - Clip ให้อยู่ในช่วงที่เหมาะสม

### 🚀 Next Step:

ตอนนี้ข้อมูลสะอาดแล้ว! แต่ยังไม่พร้อมเทรนโมเดล...

👉 [บทต่อไป: Data Transformation (Normalization & Standardization)](03_data_transformation.ipynb)

เราจะทำให้ข้อมูลอยู่ใน scale เดียวกัน!

---

## 💡 Key Takeaways

### ✅ Golden Rules:

1. **Always check data quality first!**
   - Missing values?
   - Outliers?
   - Inconsistencies?

2. **Visualize before and after cleaning**
   - Box plots for outliers
   - Histograms for distribution
   - Heatmaps for missing values

3. **Use domain knowledge**
   - ค่าอะไรที่เป็นไปไม่ได้?
   - ค่าอะไรที่ผิดปกติ?
   - ช่วงที่สมเหตุสมผลคือเท่าไร?

4. **Document your cleaning steps**
   - ทำอะไรบ้าง?
   - ทำไมถึงเลือกวิธีนั้น?
   - ผลกระทบต่อข้อมูลยังไง?

### ⚠️ Common Mistakes:

❌ Drop ข้อมูลทิ้งมากเกินไป (เสีย information)
❌ ใช้ mean fill ทั้งที่มี outliers เยอะ
❌ ไม่ตรวจสอบ business logic
❌ Clean ข้อมูลโดยไม่ visualize